In [1]:
import pandas as pd
import numpy as np
import requests
import cfbd
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#home_id, away_id

In [3]:
from __future__ import print_function
import time
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'RRRpRRQDqVoLX6S9Oivl7G9fXmZ/4TDcL9q+xg6pBW+yWfU1PIrYm/cbwKVuIfwn'
configuration.api_key_prefix['Authorization'] = 'Bearer'

# create an instance of the API class
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
try:
    # Games
    games = api_instance.get_games(year=2022)
    #pprint(games)
except ApiException as e:
    print("Exception when calling BettingApi->get_lines: %s\n" % e)

In [4]:
df = pd.DataFrame.from_records([dict(game_id=g.id,week=g.week,home_team=g.home_team, home_team_id=g.home_id, home_points=g.home_points, home_points_q=g.home_line_scores,away_points_q=g.away_line_scores,away_team_id=g.away_id, away_team=g.away_team, away_points=g.away_points,date=g.start_date,venue_id=g.venue_id,venue=g.venue,neutral_site=g.neutral_site) for g in games])
df = df[(df['home_points_q'].notnull()) & (df['away_points_q'].notnull())].copy()
df.head()

,game_id,week,home_team,home_team_id,home_points,home_points_q,away_points_q,away_team_id,away_team,away_points,date,venue_id,venue,neutral_site
0,401426532,1,Western Kentucky,98,38.0,"[7, 14, 0, 17]","[10, 7, 3, 7]",2046,Austin Peay,27.0,2022-08-27T16:00:00.000Z,3796.0,Houchens Industries-L. T. Smith Stadium,False
1,401405059,1,Northwestern,77,31.0,"[3, 14, 7, 7]","[7, 7, 14, 0]",158,Nebraska,28.0,2022-08-27T16:30:00.000Z,3504.0,Aviva Stadium,True
2,401439512,1,Florida Memorial University,125762,39.0,"[13, 13, 6, 7]","[0, 0, 21, 13]",2206,Edward Waters,34.0,2022-08-27T19:00:00.000Z,NaN,None,True
3,401413256,1,UNLV,2439,52.0,"[10, 35, 0, 7]","[7, 0, 7, 7]",304,Idaho State,21.0,2022-08-27T19:30:00.000Z,6501.0,Allegiant Stadium,False
4,401413213,1,Jacksonville State,55,42.0,"[7, 14, 21, 0]","[7, 10, 0, 0]",2617,Stephen F. Austin,17.0,2022-08-27T19:30:00.000Z,3689.0,Cramton Bowl,True


In [5]:
df = df[df['week'] < 13].copy()

In [6]:
home_df = df[['game_id','home_team_id','home_team','home_points','home_points_q','date','venue_id','venue','neutral_site','away_team','away_points','away_points_q','away_team_id']]
home_df.rename(columns={'away_team':'opponent','away_points':'points_allowed','away_points_q':'opponent_points_q','away_team_id':'opponent_id'},inplace=True)
home_points_split = home_df[['game_id','home_points_q']].copy()

/Users/kylekidder/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [7]:
home_points_split = home_df[['game_id','home_points_q']].copy()

In [8]:
split_df_home = pd.DataFrame(home_points_split['home_points_q'].to_list())

In [9]:
split_df_home.rename(columns={0:'home_points_q1',1:'home_points_q2',2:'home_points_q3',3:'home_points_q4'},inplace=True)
split_df_home = split_df_home[['home_points_q1','home_points_q2','home_points_q3','home_points_q4']].copy()

In [10]:
opponent_points_split = home_df[['game_id','opponent_points_q']].copy()

In [11]:
split_df_opponent = pd.DataFrame(opponent_points_split['opponent_points_q'].to_list())

In [12]:
split_df_opponent.rename(columns={0:'points_allowed_q1',1:'points_allowed_q2',2:'points_allowed_q3',3:'points_allowed_q4'},inplace=True)
split_df_opponent = split_df_opponent[['points_allowed_q1','points_allowed_q2','points_allowed_q3','points_allowed_q4']].copy()

In [13]:
split_df_home.head()

,home_points_q1,home_points_q2,home_points_q3,home_points_q4
0,7.0,14.0,0.0,17.0
1,3.0,14.0,7.0,7.0
2,13.0,13.0,6.0,7.0
3,10.0,35.0,0.0,7.0
4,7.0,14.0,21.0,0.0


In [14]:
split_df_opponent.head()

,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4
0,10.0,7.0,3.0,7.0
1,7.0,7.0,14.0,0.0
2,0.0,0.0,21.0,13.0
3,7.0,0.0,7.0,7.0
4,7.0,10.0,0.0,0.0


In [15]:
home_info = pd.concat([home_df,split_df_home,split_df_opponent],axis=1)
home_info['is_home'] = 1
home_info.rename(columns={'home_team_id':'team_id','home_points':'points','home_points_q1':'points_q1','home_points_q2':'points_q2','home_points_q3':'points_q3','home_points_q4':'points_q4','home_team':'school'},inplace=True)
home_info.drop(columns=['home_points_q','opponent_points_q'],axis=1,inplace=True)
home_info.head()

,game_id,team_id,school,points,date,venue_id,venue,neutral_site,opponent,points_allowed,opponent_id,points_q1,points_q2,points_q3,points_q4,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4,is_home
0,401426532.0,98.0,Western Kentucky,38.0,2022-08-27T16:00:00.000Z,3796.0,Houchens Industries-L. T. Smith Stadium,False,Austin Peay,27.0,2046.0,7.0,14.0,0.0,17.0,10.0,7.0,3.0,7.0,1
1,401405059.0,77.0,Northwestern,31.0,2022-08-27T16:30:00.000Z,3504.0,Aviva Stadium,True,Nebraska,28.0,158.0,3.0,14.0,7.0,7.0,7.0,7.0,14.0,0.0,1
2,401439512.0,125762.0,Florida Memorial University,39.0,2022-08-27T19:00:00.000Z,NaN,None,True,Edward Waters,34.0,2206.0,13.0,13.0,6.0,7.0,0.0,0.0,21.0,13.0,1
3,401413256.0,2439.0,UNLV,52.0,2022-08-27T19:30:00.000Z,6501.0,Allegiant Stadium,False,Idaho State,21.0,304.0,10.0,35.0,0.0,7.0,7.0,0.0,7.0,7.0,1
4,401413213.0,55.0,Jacksonville State,42.0,2022-08-27T19:30:00.000Z,3689.0,Cramton Bowl,True,Stephen F. Austin,17.0,2617.0,7.0,14.0,21.0,0.0,7.0,10.0,0.0,0.0,1


In [16]:
################################################################

In [17]:
away_df = df[['game_id','away_team_id','away_team','away_points','away_points_q','date','venue_id','venue','neutral_site','home_team','home_points','home_team_id']]

In [18]:
away_points_split = away_df[['game_id','away_points_q']].copy()

In [19]:
split_df_away = pd.DataFrame(away_points_split['away_points_q'].to_list())

In [20]:
split_df_away.rename(columns={0:'away_points_q1',1:'away_points_q2',2:'away_points_q3',3:'away_points_q4'},inplace=True)
split_df_away = split_df_away[['away_points_q1','away_points_q2','away_points_q3','away_points_q4']].copy()

In [21]:
home_opponent_scores = split_df_home.copy()
home_opponent_scores.rename(columns={'home_points_q1':'points_allowed_q1','home_points_q2':'points_allowed_q2','home_points_q3':'points_allowed_q3','home_points_q4':'points_allowed_q4'},inplace=True)

In [22]:
away_info = pd.concat([away_df,split_df_away,home_opponent_scores],axis=1)
away_info['is_home'] = 0
away_info.rename(columns={'away_team_id':'team_id','away_points':'points','away_points_q1':'points_q1','away_points_q2':'points_q2','away_points_q3':'points_q3','away_points_q4':'points_q4','away_team':'school','home_points':'points_allowed','home_team':'opponent','home_team_id':'opponent_id'},inplace=True)
away_info.drop(columns=['away_points_q'],axis=1,inplace=True)
away_info.head()

,game_id,team_id,school,points,date,venue_id,venue,neutral_site,opponent,points_allowed,opponent_id,points_q1,points_q2,points_q3,points_q4,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4,is_home
0,401426532.0,2046.0,Austin Peay,27.0,2022-08-27T16:00:00.000Z,3796.0,Houchens Industries-L. T. Smith Stadium,False,Western Kentucky,38.0,98.0,10.0,7.0,3.0,7.0,7.0,14.0,0.0,17.0,0
1,401405059.0,158.0,Nebraska,28.0,2022-08-27T16:30:00.000Z,3504.0,Aviva Stadium,True,Northwestern,31.0,77.0,7.0,7.0,14.0,0.0,3.0,14.0,7.0,7.0,0
2,401439512.0,2206.0,Edward Waters,34.0,2022-08-27T19:00:00.000Z,NaN,None,True,Florida Memorial University,39.0,125762.0,0.0,0.0,21.0,13.0,13.0,13.0,6.0,7.0,0
3,401413256.0,304.0,Idaho State,21.0,2022-08-27T19:30:00.000Z,6501.0,Allegiant Stadium,False,UNLV,52.0,2439.0,7.0,0.0,7.0,7.0,10.0,35.0,0.0,7.0,0
4,401413213.0,2617.0,Stephen F. Austin,17.0,2022-08-27T19:30:00.000Z,3689.0,Cramton Bowl,True,Jacksonville State,42.0,55.0,7.0,10.0,0.0,0.0,7.0,14.0,21.0,0.0,0


In [23]:
game_scoring_2022 = pd.concat([home_info,away_info],axis=0)

In [24]:
game_scoring_2022[game_scoring_2022['school']=='Alabama']

,game_id,team_id,school,points,date,venue_id,venue,neutral_site,opponent,points_allowed,opponent_id,points_q1,points_q2,points_q3,points_q4,points_allowed_q1,points_allowed_q2,points_allowed_q3,points_allowed_q4,is_home
305,401403854.0,333.0,Alabama,55.0,2022-09-03T23:30:00.000Z,3657.0,Bryant Denny Stadium,False,Utah State,0.0,328.0,17.0,24.0,14.0,0.0,0.0,0.0,0.0,0.0,1
845,401403880.0,333.0,Alabama,63.0,2022-09-17T20:00:00.000Z,3657.0,Bryant Denny Stadium,False,Louisiana Monroe,7.0,2433.0,28.0,7.0,14.0,14.0,0.0,7.0,0.0,0.0,1
1231,401403892.0,333.0,Alabama,55.0,2022-09-24T23:30:00.000Z,3657.0,Bryant Denny Stadium,False,Vanderbilt,3.0,238.0,14.0,17.0,10.0,14.0,3.0,0.0,0.0,0.0,1
1844,401403909.0,333.0,Alabama,24.0,2022-10-09T00:00:00.000Z,3657.0,Bryant Denny Stadium,False,Texas A&M,20.0,245.0,0.0,17.0,7.0,0.0,0.0,14.0,3.0,3.0,1
2440,401403922.0,333.0,Alabama,30.0,2022-10-22T23:00:00.000Z,3657.0,Bryant Denny Stadium,False,Mississippi State,6.0,344.0,14.0,14.0,7.0,14.0,0.0,13.0,0.0,7.0,1
3408,401403946.0,333.0,Alabama,34.0,2022-11-19T17:00:00.000Z,3657.0,Bryant Denny Stadium,False,Austin Peay,0.0,2046.0,14.0,3.0,0.0,7.0,0.0,7.0,7.0,7.0,1
367,401403868.0,333.0,Alabama,20.0,2022-09-10T16:00:00.000Z,3910.0,Darrell K Royal-Texas Memorial Stadium,False,Texas,19.0,251.0,10.0,0.0,0.0,10.0,3.0,7.0,3.0,6.0,0
1461,401403902.0,333.0,Alabama,49.0,2022-10-01T19:30:00.000Z,3887.0,Donald W. Reynolds Razorback Stadium Frank Bro...,False,Arkansas,26.0,8.0,14.0,14.0,0.0,21.0,0.0,7.0,16.0,3.0,0
2069,401403921.0,333.0,Alabama,49.0,2022-10-15T19:30:00.000Z,3853.0,Neyland Stadium,False,Tennessee,52.0,2633.0,7.0,13.0,15.0,14.0,21.0,7.0,6.0,18.0,0
3045,401403934.0,333.0,Alabama,31.0,2022-11-05T23:00:00.000Z,3958.0,Tiger Stadium,False,LSU,32.0,99.0,14.0,17.0,0.0,3.0,7.0,3.0,7.0,10.0,0


In [25]:
game_scoring_2022.to_csv('game_scoring_2022.csv')